In [0]:
from google.colab import drive
drive.mount('/content/drive')


KeyboardInterrupt



In [1]:
!git clone https://github.com/amazingclaude/Fake_News_Stance_Detection.git

Cloning into 'Fake_News_Stance_Detection'...
remote: Enumerating objects: 184, done.
remote: Total 184 (delta 0), reused 0 (delta 0), pack-reused 184
Receiving objects: 100% (184/184), 46.04 MiB | 32.67 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [0]:
pip install pyemd pandas gensim nltk scipy sklearn

In [2]:
!mv /content/pred.py /content/Fake_News_Stance_Detection/!mkdir /content/Fake_News_Stance_Detection/google_news
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz" -O /content/Fake_News_Stance_Detection/google_news/GoogleNews-vectors-negative300.bin.gz

--2019-11-18 18:02:01--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.86.197
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.86.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/content/Fake_News_Stance_Detection/google_news/GoogleNews-vectors-negative300.bin.gz’

/content/Fake_News_ 100%[===================>]   1.53G  82.3MB/s    in 17s     

2019-11-18 18:02:19 (92.9 MB/s) - ‘/content/Fake_News_Stance_Detection/google_news/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
!wget http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip -O /content/Fake_News_Stance_Detection/google_news/glove.840B.300d.zip

--2019-11-18 14:04:52--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘/content/Fake_News_Stance_Detection/google_news/glove.840B.300d.zip’

/content/Fake_News_ 100%[===================>]   2.03G  1.92MB/s    in 16m 57s 

2019-11-18 14:21:49 (2.04 MB/s) - ‘/content/Fake_News_Stance_Detection/google_news/glove.840B.300d.zip’ saved [2176768927/2176768927]



In [0]:
!unzip /content/Fake_News_Stance_Detection/google_news/glove.840B.300d.zip -d /content/Fake_News_Stance_Detection/google_news/

Archive:  /content/Fake_News_Stance_Detection/google_news/glove.840B.300d.zip
  inflating: /content/Fake_News_Stance_Detection/google_news/glove.840B.300d.txt  


In [0]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [0]:
!touch /content/Fake_News_Stance_Detection/google_news/model.txt

In [0]:
tmp_file = get_tmpfile("/content/Fake_News_Stance_Detection/google_news/model.txt")

In [0]:
_ = glove2word2vec('/content/Fake_News_Stance_Detection/google_news/glove.840B.300d.txt', tmp_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
!unrar x /content/Fake_News_Stance_Detection/Dataset/Dataset.rar /content/Fake_News_Stance_Detection/

In [3]:
cd /content/Fake_News_Stance_Detection/

/content/Fake_News_Stance_Detection


In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
!python pred.py

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
mode (load / train)? load
train_keep_prob= 0.6 clip_ratio= 5 batch_size_train= 500


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2019-11-1

In [0]:
cd ./..

/content/Fake_News_Stance_Detection


In [0]:
from csv import DictReader
from csv import DictWriter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from refuting import refuting_features_title,refuting_features_body
from refuting import mutual_information_title,mutual_information_body
from word2vec import word2vec_cos, wmd_distance
import gensim

#Load the Google News Word2vec model
model= gensim.models.KeyedVectors.load_word2vec_format('./google_news/GoogleNews-vectors-negative300.bin',limit=500000, binary=True)
index2word_set = set(model.wv.index2word)
for i in index2word_set:
  print(i)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  app.launch_new_instance()


Carrie_Muskat_MLB.com
Anadarko_Petroleum_Corp.
Information_Display_SID
statics
musical_revues
teenage_prodigy
Zareen
Enemy_Territory_Quake_Wars
Cottage_Grove
calming_influence
Infanta
deminers
Nam_Tai_Electronics
boast
Buoniconti
##st_Century_Breakdown
Turfgrass
MacQueen
cardiac_dysfunction
atelier
AGD
popcorn_flick
Joseph_Ngwenya
Denny_PBA
GARDEN
Human_Cloning
Yuk
Explanations
spokesman_Matthew_Hiltzik
Sheldon
spasm
LWP
Shoring
McIngvale
Toasted
Sampling_error
Amazon_Unbox
Effiong
Frosty_Wooldridge
Walkers
Quintin
ME2
Ship_Inn
Bhojshala
XPRT
Aoki
FEN
artless
ALONG_THE
Wierzba
priestly_ministry
agricultural_residues
strung
Reconstruction
Wiest
Calvin_Klein_Jeans
Jessica_Lall
deflects
Mason_Plumlee
Puli
ValueVision
toaster_oven
Arotech_Corporation
Pyxis_Mobile
Rafael_Peralta
Gasprom
missives
Gusher
Kenneth_Branagh
Lexus_IS###
6TH
DIPP
Deininger
--------------_Operating
tells_Billboard.com
NCR_Corporation
Md
Agata
MRB
muhurat
Scrubber
Disguise
daugher
Luis_Terrero
Col._Philip_Aguer
-----

KeyboardInterrupt: ignored

In [0]:
!mv /content/pred.py /content/Fake_News_Stance_Detection/

In [0]:
!mv /content/score.py /content/Fake_News_Stance_Detection/